In [37]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
import numpy as np

In [38]:
from google.colab import drive

drive.mount("/content/gdrive")
d1 = '/content/gdrive/My Drive/week 4/sentiment analysis_test.csv'
d2='/content/gdrive/My Drive/week 4/sentiment analysis_train.csv'



import chardet

def read_csv(filename):
  with open(filename, 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']
  return pd.read_csv(filename, encoding=encoding)



train_data = read_csv(d2)
test_data = read_csv(d1)

train_size = int(len(train_data) * 0.8)
test_size = len(train_data) - train_size
train_data, valid_data = train_data[0:train_size], train_data[train_size:len(train_data)]

def tokenize(sentence):
  tokens = nltk.word_tokenize(sentence)
  return tokens

import nltk
nltk.download('punkt')

train_data['tokens'] = train_data['Sentence'].apply(tokenize)
test_data['tokens'] = test_data['Sentence'].apply(tokenize)
valid_data['tokens'] = valid_data['Sentence'].apply(tokenize)
import nltk
nltk.download('stopwords')

stop_words = stopwords.words('english')

def remove_stopwords(tokens):
  filtered_tokens = [token for token in tokens if token not in stop_words]
  return filtered_tokens

train_data['filtered_tokens'] = train_data['tokens'].apply(remove_stopwords)
test_data['filtered_tokens'] = test_data['tokens'].apply(remove_stopwords)
valid_data['filtered_tokens'] = valid_data['tokens'].apply(remove_stopwords)


lemmatizer = WordNetLemmatizer()

def lemmatize(tokens):
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  return lemmatized_tokens
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


train_data['lemmatized_tokens'] = train_data['filtered_tokens'].apply(lemmatize)
test_data['lemmatized_tokens'] = test_data['filtered_tokens'].apply(lemmatize)
valid_data['lemmatized_tokens'] = valid_data['filtered_tokens'].apply(lemmatize)

from nltk.classify import NaiveBayesClassifier

def get_features(tokens):
  features = {}
  for token in tokens:
    features[token] = True
  return features

train_features = [(get_features(row['lemmatized_tokens']), row['Sentiment']) for index, row in train_data.iterrows()]
valid_features = [(get_features(row['lemmatized_tokens']), row['Sentiment']) for index, row in valid_data.iterrows()]
test_features = [(get_features(row['lemmatized_tokens']))for index,row in test_data.iterrows()]


classifier = NaiveBayesClassifier.train(train_features)
predictions = [classifier.classify(row[0]) for row in valid_features]

f1 = f1_score(valid_data['Sentiment'], predictions, average='micro')
print("F1 score:", f1)
# valid_features.row()
# test_features
predictions2 = [classifier.classify(row) for row in test_features]


prediction_df = pd.DataFrame({'Sentence': test_data['Sentence'], 'Prediction': predictions2})
prediction_df.to_csv('predictions_ps2.csv', index=False)





Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


F1 score: 0.6628982528263104
